In [1]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from work.utils.dataset import PandasDataset, MultiColorSpaceTransform
from work.utils.models import EfficientNetMultiColor as EfficientNet
from work.utils.metrics import evaluation
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import albumentations

In [2]:
output_dimensions = 5
data_dir = '../../dataset'
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv(f"../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: '/home/woshington/Projects/Doutorado/work/efficientnet-b0-08094119.pth'
}

In [3]:
print("Cuda", device)

Cuda cuda


In [4]:
val_transform =albumentations.Compose([
    MultiColorSpaceTransform()
])

In [5]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [6]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        "models/with-noise-mult-color.pth",
        weights_only=True
    )
)

response_0 = evaluation(model, dataloader, device)
print("without treatment", response_0[0])
cm_0 = confusion_matrix(response_0[1][1], response_0[1][0])

Loaded pretrained weights for efficientnet-b0


100%|██████████| 796/796 [38:09<00:00,  2.88s/it]


without treatment {'val_acc': {'mean': 55.816206109523776, 'std': 1.2407411868460452, 'ci_5': 53.706032037734985, 'ci_95': 57.85176157951355}, 'val_kappa': {'mean': 0.8277237165457386, 'std': 0.010480998576126116, 'ci_5': 0.8106696457045837, 'ci_95': 0.8447736512074808}, 'val_f1': {'mean': 0.5143755188882351, 'std': 0.013100866642445069, 'ci_5': 0.49307293742895125, 'ci_95': 0.5365917235612869}, 'val_recall': {'mean': 0.5124130220413208, 'std': 0.013136606185723704, 'ci_5': 0.49065131694078445, 'ci_95': 0.5347189158201218}, 'val_precision': {'mean': 0.5359026311337948, 'std': 0.013078489577956003, 'ci_5': 0.514653792977333, 'ci_95': 0.5572172313928604}}
